**Implementación del algoritmo RSA** 

# Especificación del problema

RSA permite el envío de mensajes codificados entre dos actores, garantizando que solo el receptor puede descifrar el mensaje y que el emisor puede cifrar el mensaje sólo si conoce una llave determinada, la cual es única para el receptor. Este algoritmo se puede utilizar para mantener la privacidad de un mensaje en escenarios donde existen atacantes que se beneficiarían de conocer el contenido de este, en la actualidad este escenario abarca desde transacciones financieras hasta la defensa de una nación.

# Desarrollo del algoritmo

El desarrollo del algoritmo tiene tres pasos importantes: la generación de claves, el encriptado del mensaje y el desencriptado del mensaje. En primer lugar se realiza la generación de las claves pública y privada del receptor del mensaje; usando la clave pública el emisor puede encriptar su mensaje de manera que solo el receptor puede leerlo usando su clave privada. Para crear las claves se siguen los siguientes pasos:


1.   Se eligen dos números primos $p$ y $q$ aleatorios de similar longitud de bits.
2.   Se calcula $n = p*q$, el cual se utilizará como el módulo para realizar las operaciones de aritmética modular necesarias en el proceso. Tanto el emisor como el receptor conocerán este valor.

3.   Haciendo uso de la función $\phi$ de Euler se calcula $\phi (n) = (p-1)*(q-1)$. EsTo es posible gracias a las propiedades de la función $\phi$ de Euler, la cual se define como la cantidad de números coprimos a n. Se define que si $p$ es un número primo, $\phi (p) = p-1$, debido a que solo el 1 es divisor. Así mismo, se dice que si $p$ y $q$ son coprimos, $\phi (pq) = \phi (p) * \phi (q)$, debido a que la descomposición en primos de $pq$ es igual a la unión de las descomposiciones de $p$ y $q$

4.   Se escoge un exponente de la clave pública $e$, el cual debe ser menor y coprimo a $\phi (n)$

5.   Se determina un exponente de la clave privada $d$ que satisfaga la congruencia $e*d = 1 \ mod(\phi (n))$

De esta forma se obtienen la clave pública $(n,e)$ y la clave privada $(n,d)$

Para realizar el cifrado se requiere el uso de la clave pública y la conversión del mensaje a un número $m$ menor y coprimo con $n$. Si estas condiciones no se cumplen no es posible garantizar la integridad del mensaje tras el proceso de descifrado.Lugo se realiza el cálculo del texto cifrado $c$ con la operación $$c=m^e \ mod(n)$$
El descifrado se realiza utilizando la clave privada sobre el texto cifrado de la siguiente forma: $$m=c^d \ mod(n)$$
Para mostrar el funcionamiento del descifrado se reemplaza $c$ en la ecuación anterior y se obtiene: $$c^d = (m^e)^d = m^{ed} \ mod(n)$$
Debido a la elección de $e$ y $d$, se tiene que $e*d=1 \ mod(\phi (n))$, lo que significa que $e*d=1+k*\phi (n)$. Reemplazando $ed$ en la ecuación anterior se obtiene: $$m^{ed} = m^{1+k*\phi (n)} = m*(m^{\phi (n)})^k = m \ mod(n)$$
En la ecuación anterior se usa el teorema de Euler en el ultimo paso. Este teorema dice que $a^{\phi (n)} = 1 \ mod(n)$, donde $a$ y $n$ son coprimos. La prueba de este teorema se realiza en los siguientes pasos:

1.   Se considera el conjunto P de números coprimos y menores que $n$.

2.   Se forma el conjunto Q multiplicando cada elemento del conjunto P por $a$

3.   Se verifica que cada elemento del conjunto P es congruente con algún elemento del conjunto Q en el módulo $n$

4.   Se calcula $u$ como el producto de los elementos de P y $v$ como el producto de los elementos de Q.

5.   Se comprueba que $v=u \ mod(n)$ debido a que sus factores son congruentes.

6.   Se verifica que $v=u*a^{\phi (n)}$, debido a que cada factor de $v$ se compone de un factor de $u$ multiplicado por $a$. Se sabe por la definición del conjunto P que existen $\phi (n)$ factores de $u$ y $v$.

7.   Se reemplaza a $v$ en la congruencia del paso 5 para obtener la congrunecia $u*a^{\phi (n)}=u \ mod(n)$

8.   Se concluye que $a^{\phi (n)}=1 \ mod(n)$






# Ejemplo del algoritmo

Se toman los primos $p=61$ y $q=53$ para tener el módulo $n=3233$. Así mismo se escoge el exponente público $e=17$ y el exponente privado $d=2753$. Para encriptar el texto $123$ se calcula $123^{17} \ mod(3233) = 855$. La desencriptación del mensaje se realiza con el exponente $d$ de la siguiente forma: $855^{2753} \ mod(3233) = 123$

# Implementación del algoritmo

In [17]:
import random
import math

'''
Implementación del algoritmo de Euclides
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

'''
Algoritmo extendido de Euclides para inversos multiplicativos
'''
def InversoMultiplicativo(e, phi):
    for d in range(phi):
      x = (e*d)%phi
      if x==1:
        return d
    return -1

'''
Prueba de primalidad.
'''
def esPrimo(num):
  if num%2==0:
    return False
  for j in range(1,10000):
    a=random.randrange(2,num-2)
    r=pow(a,math.floor((num-1)/2),num)
    if r!=1 and r!=(num-1):
      return False
    s=a/num
    if r%num !=s%num:
      return False
  return True


def generarPareja(p, q):
    if (esPrimo(p) and esPrimo(q)):
        raise ValueError('Los números deben ser primos.')
    elif p == q:
        raise ValueError('p y q no pueden ser iguales')
    #n = pq
    n = p * q

    #Se calcula la función phi de Euler de n
    phi = (p-1) * (q-1)

    #Se escoge un entero e (exponente público)
    e = random.randrange(1, phi)

    #Se verifica que e y phi son coprimos, en caso contrario se escoge un nuevo e
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Se usa el algoritmo extendido de Euclides para encontrar d (exponente privado)
    d = InversoMultiplicativo(e, phi)
    
    #Regresa las llaves públicas y privadas (pareja de exponente y módulo)
    return ((e, n), (d, n))

def encriptar(pk, texto):
    #Se obtienen los componentes de la llave
    llave, n = pk
    #Se convierte cada caracter del texto usando a^llave mod(n)
    cifrado = [(ord(char) ** llave) % n for char in texto]
    #Regresa un arreglo cifrfado
    return cifrado

def desencriptar(pk, textoCifrado):
    #Se obtienen los componentes de la llave
    llave, n = pk
    #Genera el texto usando a^llave mod n en los componentes del arreglo cifrado
    textoDescifrado = [chr((char ** llave) % n) for char in textoCifrado]
    #Regresa el texto descifrado como una cadena de caracteres
    return ''.join(textoDescifrado)
    

if __name__ == '__main__':
    
    p = int(input("Ingrese un número primo (17, 19, 23, etc): "))
    q = int(input("Ingrese un número primo distinto: "))
    print ("Generando llaves . . .")
    publica, privada = generarPareja(p, q)
    print ("La llave pública es ", publica ," y la llave privada es ", privada)
    ms = input("Ingrese un mensaje a encriptar ")
    msEncriptado = encriptar(publica, ms)
    print ("El mensaje encriptado es ")
    print (''.join(map(lambda x: str(x), msEncriptado)))
    print ("Desencriptando mensaje . . .")
    print ("El mensaje es: ")
    print (desencriptar(privada, msEncriptado))

Ingrese un número primo (17, 19, 23, etc): 61
Ingrese un número primo distinto: 53
Generando llaves . . .
La llave pública es  (2723, 3233)  y la llave privada es  (2507, 3233)
Ingrese un mensaje a encriptar crossing swords
El mensaje encriptado es 
13144031942057205712181204166230712057174219440318182057
Desencriptando mensaje . . .
El mensaje es: 
crossing swords


# Referencias

https://es.wikipedia.org/wiki/RSA , https://es.wikipedia.org/wiki/Teorema_de_Euler , https://es.wikipedia.org/wiki/Funci%C3%B3n_%CF%86_de_Euler 